In [1]:
import torch
from src.models.video_cav_mae import *
from collections import OrderedDict

input_weight_path = "/path/to/stage-2.pth"
output_weight_path = "/path/to/stage-3.pth"

stage1_weight = torch.load(input_weight_path)

cavmae_ft = VideoCAVMAEFT()
cavmae_ft = torch.nn.DataParallel(cavmae_ft)

In [2]:
stage2_weight = OrderedDict()
for k in stage1_weight.keys():
    if ('mlp' in k and ('a2v' in k or 'v2a' in k)) or 'decoder' in k:
        continue
    stage2_weight[k] = stage1_weight[k]

In [3]:
missing, unexpected = cavmae_ft.load_state_dict(stage2_weight, strict=False)
missing, unexpected

(['module.a2v.mlp.linear.weight',
  'module.a2v.mlp.linear.bias',
  'module.v2a.mlp.linear.weight',
  'module.v2a.mlp.linear.bias',
  'module.mlp_vision.weight',
  'module.mlp_vision.bias',
  'module.mlp_audio.weight',
  'module.mlp_audio.bias',
  'module.mlp_head.fc1.weight',
  'module.mlp_head.fc1.bias',
  'module.mlp_head.fc2.weight',
  'module.mlp_head.fc2.bias',
  'module.mlp_head.fc3.weight',
  'module.mlp_head.fc3.bias'],
 [])

In [4]:
torch.save(cavmae_ft.state_dict(), output_weight_path)